# Generador de Imágenes de CGR a partir de archivos .fasta

In [3]:
%%time
import time
import os
import numpy as np
import datashader as ds
import datashader.transfer_functions as tf
from datashader import reductions
import pandas as pd
# colormaps
from colorcet import fire
from matplotlib.cm import viridis
from bokeh.palettes import GnBu9
from matplotlib.cm import jet

def generar_imagenes_fasta(ruta_fasta, ruta_imagenes, colormap=GnBu9, colormap_string="GnBu9", transformation="log"):
    """
    Lee archivos .contigs.fasta desde la ruta especificada, omite la primera línea de cada archivo,
    genera una imagen basada en el Juego del Caos y guarda las imágenes en la carpeta de destino.

    Args:
        ruta_fasta (str): Directorio donde se encuentran los archivos .contigs.fasta.
        ruta_imagenes (str): Directorio donde se guardarán las imágenes generadas.
        Opcionales:
        colormap: Valor por default GnBu9. Paleta de colores de los puntos.
        colormap_string (str): Valor por default "GnBu9". Nombre de paleta de colores para nombrar archivo.
        transformation (str): Valor por default "log". Función de asignación de colores y para nombrar archivo.
    """
    
    # Crear la carpeta destino si no existe
    if not os.path.exists(ruta_imagenes):
        os.makedirs(ruta_imagenes)

    # Función para leer archivo .contigs.fasta y extraer la secuencia de ADN
    def leer_fasta(filename):
        with open(filename, 'r') as f:
            lines = f.readlines()
        # Ignorar la primera línea (cabecera) y unir el resto en una única secuencia
        secuencia = ''.join([line.strip() for line in lines[1:]])
        return secuencia

    # Función para calcular los puntos del juego del caos
    def calcular_puntos(secuencia, size=1800):
        # Diccionario que asigna coordenadas a cada nucleótido
        vertices = {'A': (0, 0), 'C': (0, size - 1), 'G': (size - 1, 0), 'T': (size - 1, size - 1)}

        # Punto inicial en el centro de la imagen
        punto = np.array([size // 2, size // 2])

        # Lista para almacenar los puntos
        puntos = []

        # Para cada nucleótido en la secuencia, actualizamos el punto y lo agregamos a la lista
        for nuc in secuencia:
            if nuc in vertices:
                vertice = np.array(vertices[nuc])
                # Punto nuevo es el promedio entre el punto actual y el vértice correspondiente
                punto = (punto + vertice) / 2
                puntos.append((punto[0], punto[1]))
                #puntos = puntos[np.isfinite(puntos['x']) & np.isfinite(puntos['y'])]

        return puntos

    # Función para generar la imagen usando datashader
    def generar_imagen(puntos, size=1800):
        # Crear un DataFrame con los puntos
        df = pd.DataFrame(puntos, columns=['x', 'y'])

        # Definir el lienzo de datashader
        canvas = ds.Canvas(plot_width=size, plot_height=size, x_range=(0, size), y_range=(0, size))

        # Agregar los puntos al lienzo usando una agregación de conteo
        agg = canvas.points(df, 'x', 'y', reductions.count())

        # Crear la imagen usando un fondo morado y puntos amarillos
        img = tf.shade(agg, cmap=colormap, how=transformation) # cmap=["#5D3FD3", "#FFFF00"]

        return img

    # Función para guardar la imagen
    #def guardar_imagen(img, filename):
    #    img.to_pil().save(filename)

    def guardar_imagen(img, filename):
        # Convertir a imagen PIL con fondo negro
        img_with_black_bg = tf.set_background(img, "black")
        
        # Guardar la imagen con fondo negro
        img_with_black_bg.to_pil().save(filename)

    # Procesar cada archivo .contigs.fasta en la carpeta
    for filename in os.listdir(ruta_fasta):
        if filename.endswith('.fasta'):  # filename.endswith('.contigs.fasta'):
            ruta_archivo = os.path.join(ruta_fasta, filename)
            print(f"Procesando {filename}...")

            # Leer secuencia del archivo .fasta
            secuencia = leer_fasta(ruta_archivo)

            # Calcular los puntos del juego del caos
            puntos = calcular_puntos(secuencia)

            # Generar la imagen
            img = generar_imagen(puntos)

            # Guardar la imagen correspondiente en la carpeta de destino
            output_filename = os.path.join(ruta_imagenes, f"{os.path.splitext(filename)[0]}_{colormap_string}_{transformation}.png")
            guardar_imagen(img, output_filename)
            print(f"Imagen guardada en {output_filename}")


CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.54 µs


In [ ]:
%%time
# Ejemplo
path_input_folder = "/files/Mariel/Tesis_Mariel/data/Examples"
path_output_folder = "/files/Mariel/Tesis_Mariel/data/Examples"
generar_imagenes_fasta(path_input_folder, path_output_folder)